<a href="https://colab.research.google.com/github/kghilothia06/Time-Series-Forecasting/blob/main/grid_search_lstm_15_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grid search lstm for monthly and quarterly sales prediction

In [1]:
#Import Libraries
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_excel
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [2]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]


In [3]:
# transform list into supervised learning format
def series_to_supervised(data, n_in, n_out=1):
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	agg.dropna(inplace=True)
	return agg.values


In [4]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))


In [5]:
# difference dataset
def difference(data, order):
	return [data[i] - data[i - order] for i in range(order, len(data))]


In [6]:
# fit a model
def model_fit(train, config):
	# unpack config
	n_input, n_nodes, n_epochs, n_batch, n_diff = config
	# prepare data
	if n_diff > 0:
		train = difference(train, n_diff)
	# transform series into supervised format
	data = series_to_supervised(train, n_in=n_input)
	# separate inputs and outputs
	train_x, train_y = data[:, :-1], data[:, -1]
	# reshape input data into [samples, timesteps, features]
	n_features = 1
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
	# define model
	model = Sequential()
	model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
	model.add(Dense(n_nodes, activation='relu'))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam')
	# fit model
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model


In [7]:
# forecast with the fit model
def model_predict(model, history, config):
	# unpack config
	n_input, _, _, _, n_diff = config
	# prepare data
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]
		history = difference(history, n_diff)
	# reshape sample into [samples, timesteps, features]
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# forecast
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

In [8]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# fit model
	model = model_fit(train, cfg)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = model_predict(model, history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	print(' > %.3f' % error)
	return error

In [9]:
# score a model, return None on failure
def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)


In [10]:
# grid search configs
def grid_search(data, cfg_list, n_test):
	# evaluate configs
	scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

Quarterly Sales

In [11]:
# create a list of configs for qaurterly sales to try
def model_configs():
	# define scope of configs
  # n input: The number of prior inputs to use as input for the model.
  # n nodes: The number of nodes to use in the hidden layer.
  # n epochs: The number of training epochs.
  # n batch: The number of samples to include in each mini-batch.
  # n diff: The difference order.
	n_input = [4]
	n_nodes = [50]
	n_epochs = [20]
	n_batch = [1, 12]
	n_diff = [0]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs


In [12]:
series = read_excel('/content/Sales-quarterly.xlsx', header=0, index_col=0)
series = DataFrame(series['Sales'])
data = series.values
# data split
n_test = 4
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)

Total configs: 2
 > 282851.830
 > 310469.963
 > 345605.787
 > 232089.235
 > 315041.594
 > 235737.322
 > 243747.237
 > 171081.238
 > 259576.306
 > 257858.439
> Model[[4, 50, 20, 1, 0]] 265405.895
 > 321433.135
 > 337985.625
 > 331416.080
 > 264975.734
 > 307610.709
 > 313344.072
 > 326570.575
 > 265918.063
 > 245025.477
 > 259143.899
> Model[[4, 50, 20, 12, 0]] 297342.337
done
[4, 50, 20, 1, 0] 265405.89494165854
[4, 50, 20, 12, 0] 297342.3367696112


Monthly sales

In [13]:
# create a list of configs for qaurterly sales to try
def model_configs():
	# define scope of configs
  # n input: The number of prior inputs to use as input for the model.
  # n nodes: The number of nodes to use in the hidden layer.
  # n epochs: The number of training epochs.
  # n batch: The number of samples to include in each mini-batch.
  # n diff: The difference order.
	n_input = [4]
	n_nodes = [50]
	n_epochs = [20]
	n_batch = [1, 12]
	n_diff = [12]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs


In [14]:
series = read_excel('/content/Sales-monthly.xlsx', header=0, index_col=0)
#series = DataFrame(series['Sales'])
data = series.values
# data split
n_test = 12
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)

Total configs: 2
 > 124289.506
 > 121486.124
 > 107931.080
 > 220908.327
 > 151503.758
 > 167258.807
 > 164624.129
 > 142578.168
 > 122904.945
 > 121325.917
> Model[[4, 50, 20, 1, 12]] 144481.076
 > 68460.273
 > 72474.823
 > 64798.482
 > 73793.879
 > 67139.207
 > 64363.890
 > 71520.262
 > 70028.004
 > 70134.491
 > 65997.448
> Model[[4, 50, 20, 12, 12]] 68871.076
done
[4, 50, 20, 12, 12] 68871.07595863233
[4, 50, 20, 1, 12] 144481.07625275108
